In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [3]:
'''
Paths and parameters
'''

module_path = 'Data/US/process.py'
codex_path = 'Data/US/codex.json'
output_path = 'Data/US/Processed/'
verbose = True

'''
Loading processing module
'''

module = src.inputs.write_data.load_module(module_path)

'''
Loading data
'''

data = src.inputs.load_data.load(codex_path, verbose = verbose)

Data loaded: 79.28 seconds


In [4]:
data.keys()

dict_keys(['plants_2020', 'plants_2021', 'transmission', 'impacts_parsed', 'impacts_input', 'impacts', 'onshore_wind_generation_profile', 'load_profile', 'onshore_wind_capacity', 'onshore_wind_capital_cost', 'solar_regional_capacity', 'solar_generation_profile', 'solar_capital_cost', 'solar_capacity_factor', 'regional_cost', 'unit_cost', 'rps', 'storage_mandates', 'hydro'])

In [5]:
'''
Processing raw data
'''

module = src.inputs.write_data.load_module(module_path)

installed = module.build_installed_assets(data, verbose = verbose)
optional = module.build_optional_assets(data, verbose = verbose)
lines = module.build_lines(data, verbose = verbose)
profiles = module.build_profiles(data, verbose = verbose)
policies = module.build_policies(data, verbose = verbose)

Installed assets built: 93.99 seconds
Optional assets built: 0.02279 seconds
Lines built: 0.008597 seconds
Profiles built: 0.7158 seconds
Policies built: 0.00869 seconds


In [6]:
policies

{'rps': {'AZ': 0.086690195,
  'CA': 0.447622503,
  'CO': 0.195712274,
  'CT': 0.336294722,
  'DE': 0.175451804,
  'IA': 0.005454992,
  'IL': 0.22161836,
  'KS': 0.0,
  'MA': 0.267725755,
  'MD': 0.386638895,
  'ME': 0.520222469,
  'MI': 0.208333333,
  'MN': 0.249791434,
  'MO': 0.098480274,
  'MT': 0.0,
  'NC': 0.048230807,
  'NE': 0.0,
  'NH': 0.207723388,
  'NJ': 0.41482223,
  'NM': 0.363695731,
  'NS': 0.4,
  'NV': 0.291710789,
  'NY': 0.515187901,
  'OH': 0.060610023,
  'OR': 0.202639484,
  'PA': 0.07810129,
  'RI': 0.335916647,
  'TX': 0.039919281,
  'voluntary': 0.059794406,
  'VT': 0.63,
  'WA': 0.129821978,
  'WI': 0.100067305}}

In [7]:
'''
Formatting processed data
'''
module = src.inputs.write_data.load_module(module_path)

profiles_data, scale = module.format_profiles(
    profiles, verbose = verbose
)

installed_data = module.format_installed_assets(
    installed, scale, verbose = verbose
)

optional_data = module.format_optional_assets(
    optional, verbose = verbose
)

lines_data = module.format_lines(
    lines, verbose = verbose
)

policies_data = module.format_policies(
    policies, verbose = verbose
)

Profiles formatted: 0.1076 seconds
Installed assets formatted: 3.907 seconds
Optional assets formatted: 0.06575 seconds
Lines formatted: 0.03412 seconds
Policies formatted: 2.313e-05 seconds


In [8]:
'''
Writing JSONs
'''
module = src.inputs.write_data.load_module(module_path)

module.write_assets(
    {**installed_data, **optional_data},
    output_path = output_path, verbose = verbose
)

module.write_lines(
    lines_data, output_path = output_path, verbose = verbose
)

module.write_profiles(
    profiles_data, output_path = output_path, verbose = verbose
)

module.write_policies(
    policies_data, output_path = output_path, verbose = verbose
)

Assets written: 4.527 seconds
Lines written: 3.24 seconds
Profles written: 1.551e+03 seconds
Policies written: 0.004388 seconds
